In [1]:
import pandas as pd
import numpy as np

In [2]:
movies_df = pd.read_csv('dataset/movies.dat',header= None,sep = '::',engine='python',names = ['movie_id','movie_title','movie_genre'])

movies_df.head()


,movie_id,movie_title,movie_genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
# we convert the movie genres to set of dummy variables
movies_df  = pd.concat([movies_df,movies_df.movie_genre.str.get_dummies(sep='|')],axis = 1)
movies_df.head()

,movie_id,movie_title,movie_genre,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
movies_categories = movies_df.columns[3:]
movies_df.loc[0]

movie_id                                 1
movie_title               Toy Story (1995)
movie_genre    Animation|Children's|Comedy
Action                                   0
Adventure                                0
Animation                                1
Children's                               1
Comedy                                   1
Crime                                    0
Documentary                              0
Drama                                    0
Fantasy                                  0
Film-Noir                                0
Horror                                   0
Musical                                  0
Mystery                                  0
Romance                                  0
Sci-Fi                                   0
Thriller                                 0
War                                      0
Western                                  0
Name: 0, dtype: object

In [5]:
from collections import OrderedDict

user_preferences = OrderedDict(zip(movies_categories,[]))
user_preferences['Action'] = 5  
user_preferences['Adventure'] = 5  
user_preferences['Animation'] = 1  
user_preferences["Children's"] = 1  
user_preferences["Comedy"] = 3  
user_preferences['Crime'] = 2  
user_preferences['Documentary'] = 1  
user_preferences['Drama'] = 1  
user_preferences['Fantasy'] = 5  
user_preferences['Film-Noir'] = 1  
user_preferences['Horror'] = 2  
user_preferences['Musical'] = 1  
user_preferences['Mystery'] = 3  
user_preferences['Romance'] = 1  
user_preferences['Sci-Fi'] = 5  
user_preferences['War'] = 3  
user_preferences['Thriller'] = 2  
user_preferences['Western'] =1  
    
print(user_preferences)


OrderedDict([('Action', 5), ('Adventure', 5), ('Animation', 1), ("Children's", 1), ('Comedy', 3), ('Crime', 2), ('Documentary', 1), ('Drama', 1), ('Fantasy', 5), ('Film-Noir', 1), ('Horror', 2), ('Musical', 1), ('Mystery', 3), ('Romance', 1), ('Sci-Fi', 5), ('War', 3), ('Thriller', 2), ('Western', 1)])


In [19]:
def dot_product(v1, v2):
    return sum([i*j for i,j in zip(v1,v2)])

def get_movie_score(movie_feats, user_prefs):
    return dot_product(movie_feats, user_prefs)

In [9]:
toy_story_feats = movies_df.loc[0][movies_categories]
toy_story_feats

Action         0
Adventure      0
Animation      1
Children's     1
Comedy         1
Crime          0
Documentary    0
Drama          0
Fantasy        0
Film-Noir      0
Horror         0
Musical        0
Mystery        0
Romance        0
Sci-Fi         0
Thriller       0
War            0
Western        0
Name: 0, dtype: object

In [55]:
toy_story_user_predicted_score = dot_product(toy_story_feats,user_preferences.values())
toy_story_user_predicted_score

5

In [56]:
movies_df[movies_df.movie_title.str.contains('Die Hard')]

,movie_id,movie_title,movie_genre,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
163,165,Die Hard: With a Vengeance (1995),Action|Thriller,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1023,1036,Die Hard (1988),Action|Thriller,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1349,1370,Die Hard 2 (1990),Action|Thriller,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [57]:
die_hard_id = 1036
die_hard_feats = movies_df[movies_df.movie_id == die_hard_id][movies_categories]
die_hard_feats.T


,1023
Action,1
Adventure,0
Animation,0
Children's,0
Comedy,0
Crime,0
Documentary,0
Drama,0
Fantasy,0
Film-Noir,0


In [58]:
die_hard_predicted_score = dot_product(die_hard_feats.values[0],user_preferences.values())
die_hard_predicted_score

8

In [66]:
def get_movie_recommendations(user_prefs, n_recommendation):
    movies_df['score'] = movies_df[movies_categories].apply(get_movie_score,args=[user_prefs.values()],axis = 1)
    return movies_df.sort_values(by=['score'],ascending=False)[['movie_title','score']][:n_recommendation]
get_movie_recommendations(user_preferences, 30)

,movie_title,score
2253,Soldier (1998),20
257,Star Wars: Episode IV - A New Hope (1977),20
2036,Tron (1982),20
1197,Army of Darkness (1993),20
2559,Star Wars: Episode I - The Phantom Menace (1999),20
1985,"Honey, I Shrunk the Kids (1989)",19
1192,Star Wars: Episode VI - Return of the Jedi (1983),18
1111,"Abyss, The (1989)",18
1848,Armageddon (1998),18
2847,Total Recall (1990),18


# Collaborative Filtering

In [78]:
ratings_df = pd.read_table('dataset/ratings.dat', header = None, engine='python',sep= '::',names=['user_id','movie_id','rating','timestamp'])
del ratings_df['timestamp']
ratings_df = pd.merge(ratings_df,movies_df,on='movie_id')[['user_id','movie_title','movie_id','rating']]

ratings_df.head()

,user_id,movie_title,movie_id,rating
0,1,One Flew Over the Cuckoo's Nest (1975),1193,5
1,2,One Flew Over the Cuckoo's Nest (1975),1193,5
2,12,One Flew Over the Cuckoo's Nest (1975),1193,4
3,15,One Flew Over the Cuckoo's Nest (1975),1193,4
4,17,One Flew Over the Cuckoo's Nest (1975),1193,5


In [88]:
# make a rating metrics
ratings_mtx_df = ratings_df.pivot_table(values='rating',index ='user_id',columns='movie_title')
ratings_mtx_df.fillna(0,inplace=True)
movie_index = ratings_mtx_df.columns
ratings_mtx_df.head()



movie_title,"$1,000,000 Duck (1971)",'Night Mother (1986),'Til There Was You (1997),"'burbs, The (1989)",...And Justice for All (1979),1-900 (1994),10 Things I Hate About You (1999),101 Dalmatians (1961),101 Dalmatians (1996),12 Angry Men (1957),...,"Young Poisoner's Handbook, The (1995)",Young Sherlock Holmes (1985),Young and Innocent (1937),Your Friends and Neighbors (1998),Zachariah (1971),"Zed & Two Noughts, A (1985)",Zero Effect (1998),Zero Kelvin (Kjærlighetens kjøtere) (1995),Zeus and Roxanne (1997),eXistenZ (1999)
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [90]:
corr_matrix = np.corrcoef(ratings_mtx_df.T)  # correlation b/w movies if not T then correlation b/w users.
corr_matrix.shape


(3706, 3706)

In [109]:
favoured_movie_title = 'Toy Story (1995)'

favoured_movie_index = list(movie_index).index(favoured_movie_title)

P = corr_matrix[favoured_movie_index]

# return movies with high correlation with Toy story

list(movie_index[ (P > 0.4) & (P < 1.0)])


['Aladdin (1992)',
 "Bug's Life, A (1998)",
 'Groundhog Day (1993)',
 'Lion King, The (1994)',
 'Toy Story 2 (1999)']

In [128]:
def get_movie_similarity(movie_title):
    movie_idx = list(movie_index).index(movie_title)
    return corr_matrix[movie_idx]

def get_movie_recommendations(user_movies):
    movies_similarities = np.zeros(corr_matrix.shape[0])
    for movie_id in user_movies:
        movies_similarities = movies_similarities + get_movie_similarity(movie_id)
    similarities_df = pd.DataFrame({
        'movie_title': movie_index,
        'sum_similarity':  movies_similarities
    }) 
    similarities_df = similarities_df[~(similarities_df.movie_title.isin(user_movies))]
    similarities_df = similarities_df.sort_values(by = ['sum_similarity'], ascending = False)
    return similarities_df
    

In [140]:
sample_user = 19

ratings_df[ratings_df.user_id == sample_user].sort_values(by = ['rating'],ascending  = False)

,user_id,movie_title,movie_id,rating
6,19,One Flew Over the Cuckoo's Nest (1975),1193,5
498903,19,Christmas Vacation (1989),2423,5
219313,19,Jaws (1975),1387,5
107021,19,American Beauty (1999),2858,5
444548,19,"Jungle Book, The (1967)",2078,5
659440,19,Highlander (1986),1275,5
446705,19,Young Frankenstein (1974),1278,5
4210,19,"Bug's Life, A (1998)",2355,5
128362,19,Star Wars: Episode V - The Empire Strikes Back...,1196,5
459163,19,Star Trek: Generations (1994),329,5


In [137]:
sample_user_movies = ratings_df [ratings_df.user_id ==sample_user].movie_title.tolist()
recommendation = get_movie_recommendations(sample_user_movies)

recommendation.movie_title.head(20)


3567                                 Wayne's World (1992)
231     Austin Powers: International Man of Mystery (1...
716                                       Clueless (1995)
1426                           Grosse Pointe Blank (1997)
1275                               Full Monty, The (1997)
376                                            Big (1988)
2682                                  Pulp Fiction (1994)
1149                               Few Good Men, A (1992)
1756                                 Jerry Maguire (1996)
2247                                Mrs. Doubtfire (1993)
2705                                     Quiz Show (1994)
2517                                    Parenthood (1989)
1919                                     Liar Liar (1997)
514                            Breakfast Club, The (1985)
3013                               Sixteen Candles (1984)
3246                                      Swingers (1996)
1861                                    L.A. Story (1991)
1173          